## Make approximate maps of scan coverage at different times of the year
### A. Ordog, August 2022 

## Import packages

In [ ]:
import numpy as np
from astropy import units as u
from astropy.coordinates import SkyCoord,EarthLocation, AltAz, ICRS, Galactic, FK4, FK5
from astropy.time import Time
import matplotlib.pyplot as plt
import datetime
from astropy.wcs import WCS
import datetime
import matplotlib.dates as mdates
import ephem

## Set up survay scan parameters

In [ ]:
###################################################################

#### KEEP THESE PARAMETERS THE SAME:
nscan = 72    # number of single direction in 24 sidereal hours
vscan = 1/3   # scan speed in degrees per second
dscan = 360.  # extent of the scans in azimuth
az0 = 0.      # starting azimuth of scans

#### MODIFY THESE AS NEEDED:
# The sampling time - determines spacing between points:
tsamp = 1.0   
# Number of sets of scans - in the survey we actually have 20,
# but with smaller values it's easier to see what's going on in
# the 'simulated' plots:
N = 3 
# Starting date for scans. In reality, this should be a specific
# date for each observing night, but for the purposes of making
# approximate maps of sky coverage, a single date, sometime in the
# middle of the survey phase is fine. I used the following:
# Phase 1: '2022-06-30T00:00:00.0'
# Phase 2: '2022-09-15T00:00:00.0'
# Phase 3: '2022-12-15T00:00:00.0'
t0 = '2022-06-30T00:00:00.0'
phase = 1
##############################
###################################################################
###################################################################

# Define some other parameters (mostly conversions between time and sidereal time)

sid_to_t = (23+56/60+4.1/3600)/24
t_to_sid = 1/sid_to_t

tscan = dscan/vscan 
tscansid = tscan*t_to_sid
tsampsid = tsamp*t_to_sid
vscansid = vscan*sid_to_t
print(vscansid)

twaitsid = (24*3600-nscan*tscansid)/nscan
twait = twaitsid*sid_to_t

npts = int(np.floor(tscansid/tsampsid))

t0sid = np.empty(N)
for i in range(0,N):
    t0sid[i] = i*(twaitsid*2+tscansid*2)/N
    
print(npts)
print(tscan/60)
print(tscansid/60)
print(twait/60)
print(twaitsid/60)
print(t0sid/60)

## Calculate coordinates at each point (takes a while)

In [ ]:
loc = EarthLocation(lat = 49.3*u.deg, lon = -119.6*u.deg, height = 546*u.m)

#tsid = np.empty([nscan*N,npts])
tdelt_arr = np.empty([nscan*N,npts])
az = np.empty([nscan*N,npts])
el_pol = np.empty([nscan*N,npts])
el_20 = np.empty([nscan*N,npts])
ra1 = np.empty([nscan*N,npts])
dec1 = np.empty([nscan*N,npts])
l1 = np.empty([nscan*N,npts])
b1 = np.empty([nscan*N,npts])
ra2 = np.empty([nscan*N,npts])
dec2 = np.empty([nscan*N,npts])
l2 = np.empty([nscan*N,npts])
b2 = np.empty([nscan*N,npts])
t = []

samp_arr = np.linspace(0,(npts-1)*tsamp,npts)

for k in range(0,N):
    print('====================')
    print(k)
    scandir = 1
    az0use = az0   

    for i in range(0,nscan):
        
        tdelt_arr[k*nscan+i,:] = i*(tscan+twait)+t0sid[k]*sid_to_t+samp_arr
                            
        az[k*nscan+i,:] = scandir*(samp_arr*t_to_sid)*vscansid+az0use
        el_20[k*nscan+i,:] = 20.0
        el_pol[k*nscan+i,:] = 49.3
        t.append(Time(t0,format='isot',scale='utc')+datetime.timedelta(seconds=1)*tdelt_arr[k*nscan+i,:])

        AltAzcoord1 = SkyCoord(alt=el_pol[k*nscan+i,:]*u.deg, az=az[k*nscan+i,:]*u.deg, obstime=t[k*nscan+i][:], 
                              frame = 'altaz', location = loc)
        ra1[k*nscan+i,:]  = AltAzcoord1.icrs.ra
        dec1[k*nscan+i,:] = AltAzcoord1.icrs.dec
        l1[k*nscan+i,:]   = AltAzcoord1.galactic.l.deg
        b1[k*nscan+i,:]   = AltAzcoord1.galactic.b.deg
        
        AltAzcoord2 = SkyCoord(alt=el_20[k*nscan+i,:]*u.deg, az=az[k*nscan+i,:]*u.deg, obstime=t[k*nscan+i][:], 
                              frame = 'altaz', location = loc)
        ra2[k*nscan+i,:]  = AltAzcoord2.icrs.ra
        dec2[k*nscan+i,:] = AltAzcoord2.icrs.dec
        l2[k*nscan+i,:]   = AltAzcoord2.galactic.l.deg
        b2[k*nscan+i,:]   = AltAzcoord2.galactic.b.deg
        
        az0use = az0use + scandir*dscan
        scandir = -1*scandir   


## Make array of sidereal times for each point

In [ ]:
tsid = np.empty([nscan*N,npts])

for k in range(0,N):

    for i in range(0,nscan):
        
        tfix = Time(t[k*nscan+i][:],scale='utc',location=loc).datetime
        tsid[k*nscan+i,:] = Time(tfix,scale='utc',location=loc).sidereal_time('mean').hour


## Set up range of available sidereal times given the season:

In [ ]:
if phase == 1:
    t1 = 0  # less than 0 hrs (nothing) OR
    t2 = 13 # greater than 13 hrs
if phase == 2:
    t1 = 8  # less than 8 hrs OR
    t2 = 17 # greater than 17 hrs
if phase == 3:
    t1 = 15 # less than 15 hrs OR
    t2 = 20 # greater than 20 hrs

## Plot azimuth versus sidereal time

In [ ]:
#scanplot = np.linspace(0,nscan-1,nscan)
plt.figure(figsize=(20,4))

spts = 0.1
fs = 14

for i in range(0,N):
    for j in range(0,nscan):
        
        igood1 = np.where(tsid[i*nscan+int(j),:]<t1)
        igood2 = np.where(tsid[i*nscan+int(j),:]>t2)

        plt.scatter(tsid[i*nscan+int(j),igood1],az[i*nscan+int(j),igood1],color='C'+str(int(phase-1)),s=spts)
        plt.scatter(tsid[i*nscan+int(j),igood2],az[i*nscan+int(j),igood2],color='C'+str(int(phase-1)),s=spts)

plt.xlabel('Sidereal time (hrs)',fontsize=fs)
plt.ylabel('Azimuth (deg.)',fontsize=fs)
plt.xticks(fontsize=fs)
plt.yticks(fontsize=fs)

plt.ylim(-10,370)
plt.xlim(0,24)
plt.grid()

plt.tight_layout()
plt.savefig('../PLOTS/phase'+str(phase)+'_azimuth_vs_sidtime.jpg')

## Plot scans in Equatorial coordinates

In [ ]:
fig = plt.figure(figsize=(20,6))
plt.subplot(111, projection='rectilinear')
plt.grid(True)

spts = 0.2
fs = 14

for i in range(0,N):
    for j in range(0,nscan):
        
        igood1 = np.where(tsid[i*nscan+int(j),:]<t1)
        igood2 = np.where(tsid[i*nscan+int(j),:]>t2)
        
        eq1 = SkyCoord(ra1[i*nscan+int(j),igood1], dec1[i*nscan+int(j),igood1], frame='icrs', unit=u.deg)
        eq2 = SkyCoord(ra1[i*nscan+int(j),igood2], dec1[i*nscan+int(j),igood2], frame='icrs', unit=u.deg)
        
        eq3 = SkyCoord(ra2[i*nscan+int(j),igood1], dec2[i*nscan+int(j),igood1], frame='icrs', unit=u.deg)
        eq4 = SkyCoord(ra2[i*nscan+int(j),igood2], dec2[i*nscan+int(j),igood2], frame='icrs', unit=u.deg)
        
        # plot the high elevation scans:
        plt.scatter(eq1.ra.hour, eq1.dec.deg,s=spts,c='C'+str(int(phase-1)))
        plt.scatter(eq2.ra.hour, eq2.dec.deg,s=spts,c='C'+str(int(phase-1)))
        
        # plot the low elevation scans:
        plt.scatter(eq3.ra.hour, eq3.dec.deg,s=spts,c='C'+str(int(phase-1)))
        plt.scatter(eq4.ra.hour, eq4.dec.deg,s=spts,c='C'+str(int(phase-1)))


plt.xlim(24,0)
plt.ylim(-25,90)
plt.yticks(fontsize=fs)
plt.xlabel('Right Ascension (hours)',fontsize=fs)
plt.ylabel('Declination (degrees)',fontsize=fs)

plt.tight_layout()
plt.savefig('../PLOTS/phase'+str(phase)+'_equatorial.jpg')

## Plot scans in Galactic coordinates

In [ ]:
fig = plt.figure(figsize=(20,10))
plt.subplot(111, projection='aitoff')
plt.grid(True)

spts = 0.2
fs = 12

for i in range(0,N):
    for j in range(0,nscan):
        
        igood1 = np.where(tsid[i*nscan+int(j),:]<t1)
        igood2 = np.where(tsid[i*nscan+int(j),:]>t2)
        
        gal1 = SkyCoord(l1[i*nscan+int(j),igood1], b1[i*nscan+int(j),igood1], frame='galactic', unit=u.deg)
        gal2 = SkyCoord(l1[i*nscan+int(j),igood2], b1[i*nscan+int(j),igood2], frame='galactic', unit=u.deg)
        
        gal3 = SkyCoord(l2[i*nscan+int(j),igood1], b2[i*nscan+int(j),igood1], frame='galactic', unit=u.deg)
        gal4 = SkyCoord(l2[i*nscan+int(j),igood2], b2[i*nscan+int(j),igood2], frame='galactic', unit=u.deg)
        
        # plot the high elevation scans:
        plt.scatter(-gal1.l.wrap_at('180d').radian, gal1.b.radian,s=spts,c='C'+str(int(phase-1)))
        plt.scatter(-gal2.l.wrap_at('180d').radian, gal2.b.radian,s=spts,c='C'+str(int(phase-1)))
        
        # plot the low elevation scans:
        plt.scatter(-gal3.l.wrap_at('180d').radian, gal3.b.radian,s=spts,c='C'+str(int(phase-1)))
        plt.scatter(-gal4.l.wrap_at('180d').radian, gal4.b.radian,s=spts,c='C'+str(int(phase-1)))

positions = (-150*np.pi/180.,-120*np.pi/180.,-90*np.pi/180.,-60*np.pi/180.,-30*np.pi/180.,0,
             30*np.pi/180.,60*np.pi/180.,90*np.pi/180.,120*np.pi/180.,150*np.pi/180.)
labels = ("150","120","90","60","30","0","330","300","270","240","210")
plt.xticks(positions, labels,fontsize=fs)
plt.yticks(fontsize=fs)
plt.xlabel('Galactic Longitude (degrees)',fontsize=fs)
plt.ylabel('Galactic Latitude (degrees)',fontsize=fs)

plt.tight_layout()
plt.savefig('../PLOTS/phase'+str(phase)+'_galactic.jpg')